In [1]:
%reset -fs

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn

In [3]:
def find_speaker(text):
    return text.split(':')[0].strip()
def find_text(text):
    return text.split(':')[1]

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

In [5]:
with open('fe7_only.txt') as f:
    lines = f.read()

In [6]:
lines

'\n\n------------------------------------------------------------------------------\n  4.0  PROLOGUE : A GIRL FROM THE PLAIN\n------------------------------------------------------------------------------\n\nLyn  : Are you awake? I found you unconscious on the plains. I am Lyn, of the\n   Lorca tribe. You\'re safe now. Who are you? Can you remember your name?\n   Your name is Agung? What an odd-sounding name... But pay me no mind. It is\n   a good name. I see by your attire that you are a traveler.\n   What brings you to the Sacae Plains? Would you share your story with me?\n   Hm? What was that noise? I\'ll go see what\'s happening. Agung, wait here\n   for me.\n\nLyn  : Oh, no! Bandits! They must have come down from the Bern Mountains!\n   They must be planning on raiding the local villages. I... I have to stop\n   them! If that\'s all of them, I think I can handle them on my own. You\'ll\n   be safe in here, Agung! What? You want to help? Well, can you use a\n   weapon? Ah, I see...

In [7]:
lines = lines.replace("\n   ", " ")
lines = lines.replace("Agung", "Mark")
lines = lines.split('\n')

In [8]:
':' not in lines[6]

False

In [9]:
for i in range(len(lines) - 1, -1, -1):
    if (':' not in lines[i]) or ("CHAPTER" in lines[i]) or ("PROLOGUE" in lines[i]) or (lines[i][-1] == ":"):
        lines.pop(i)
        

In [10]:
lines

["Lyn  : Are you awake? I found you unconscious on the plains. I am Lyn, of the Lorca tribe. You're safe now. Who are you? Can you remember your name? Your name is Mark? What an odd-sounding name... But pay me no mind. It is a good name. I see by your attire that you are a traveler. What brings you to the Sacae Plains? Would you share your story with me? Hm? What was that noise? I'll go see what's happening. Mark, wait here for me.",
 "Lyn  : Oh, no! Bandits! They must have come down from the Bern Mountains! They must be planning on raiding the local villages. I... I have to stop them! If that's all of them, I think I can handle them on my own. You'll be safe in here, Mark! What? You want to help? Well, can you use a weapon? Ah, I see... So you're a strategist by trade? An odd profession, but... Very well. We'll go together!",
 "Lyn  : Over here! If you want to help, Mark, I could use your advice. I'll protect you, so stay close to me.",
 'Lyn  : I need to be closer to the enemy.',
 'L

In [11]:
df = pd.DataFrame()
df['text'] = lines
df

,text
0,Lyn : Are you awake? I found you unconscious ...
1,"Lyn : Oh, no! Bandits! They must have come do..."
2,"Lyn : Over here! If you want to help, Mark, I..."
3,Lyn : I need to be closer to the enemy.
4,"Lyn : Yes, this should be close enough."
...,...
3642,Rebecca : Forgive me... Brother...
3643,Raven : I'm done for. Is this really... the be...
3644,Sain : Lady Lyndis... I... was... so happy...
3645,Serra : Ooh! Why me? This is SO annoying!


In [12]:
df['speaker'] = lines
df['text'] = df['text'].apply(find_text)
df['speaker'] = df['speaker'].apply(find_speaker)
df

,text,speaker
0,Are you awake? I found you unconscious on the...,Lyn
1,"Oh, no! Bandits! They must have come down fro...",Lyn
2,"Over here! If you want to help, Mark, I could...",Lyn
3,I need to be closer to the enemy.,Lyn
4,"Yes, this should be close enough.",Lyn
...,...,...
3642,Forgive me... Brother...,Rebecca
3643,I'm done for. Is this really... the best I co...,Raven
3644,Lady Lyndis... I... was... so happy...,Sain
3645,Ooh! Why me? This is SO annoying!,Serra


In [13]:
stop_words_plus = ENGLISH_STOP_WORDS.union(['ah', 'yes', 'no', 'um', 'oh', 'hm', 'uh', 've', 'huh', 'eh', 'ha', 'hey',
                                           'ok', 'sure', 'll', 'let', 'isn', 'heh', 'don', 'did', 'just', 'like', 'think', 'hee', 'fine'])

In [14]:
cv = CountVectorizer(strip_accents = 'unicode', stop_words = stop_words_plus, lowercase = True)
X = cv.fit_transform(df['text'])

In [15]:
df_cv = pd.DataFrame(X.todense(), index = df['text'], columns = cv.get_feature_names())

/Users/dayvdoberne/opt/anaconda3/envs/metis/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
df_cv

,000,100,18,19,30,50,75th,aa,aaaaaaah,aaaaah,...,yo,yon,youdoing,young,younger,yourd,youre,youth,yu,zephiel
text,,,,,,,,,,,,,,,,,,,,,
"Are you awake? I found you unconscious on the plains. I am Lyn, of the Lorca tribe. You're safe now. Who are you? Can you remember your name? Your name is Mark? What an odd-sounding name... But pay me no mind. It is a good name. I see by your attire that you are a traveler. What brings you to the Sacae Plains? Would you share your story with me? Hm? What was that noise? I'll go see what's happening. Mark, wait here for me.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Oh, no! Bandits! They must have come down from the Bern Mountains! They must be planning on raiding the local villages. I... I have to stop them! If that's all of them, I think I can handle them on my own. You'll be safe in here, Mark! What? You want to help? Well, can you use a weapon? Ah, I see... So you're a strategist by trade? An odd profession, but... Very well. We'll go together!",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Over here! If you want to help, Mark, I could use your advice. I'll protect you, so stay close to me.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
I need to be closer to the enemy.,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Yes, this should be close enough.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Forgive me... Brother...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
I'm done for. Is this really... the best I could aspire to?,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Lady Lyndis... I... was... so happy...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
tfidf = TfidfVectorizer(**cv.get_params())
X_tfidf = tfidf.fit_transform(df['text'])

set_components = 6
itera = 20

lda_tf = LatentDirichletAllocation(n_components = set_components, max_iter = itera, random_state = 0)
lda_tf.fit(X)

lda_tfidf = LatentDirichletAllocation(n_components = set_components, max_iter = itera, random_state = 0)
lda_tfidf.fit(X_tfidf)

/Users/dayvdoberne/opt/anaconda3/envs/metis/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2029: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn(


LatentDirichletAllocation(max_iter=20, n_components=6, random_state=0)

In [18]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, X, cv)

/Users/dayvdoberne/opt/anaconda3/envs/metis/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/dayvdoberne/opt/anaconda3/envs/metis/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/dayvdoberne/opt/anaconda3/envs/metis/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/dayvdoberne/opt/anaconda3/envs/metis/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favo

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.070246 -0.001659       1        1  19.902131
4     -0.184549  0.004732       2        1  19.602623
3      0.020812  0.098419       3        1  16.827941
0      0.044514 -0.176135       4        1  16.569609
2      0.099262 -0.020362       5        1  14.694894
1      0.090207  0.095005       6        1  12.402802, topic_info=            Term        Freq       Total Category  logprob  loglift
1869        lord  192.000000  192.000000  Default  30.0000  30.0000
2637       right  145.000000  145.000000  Default  29.0000  29.0000
3179       thank   67.000000   67.000000  Default  28.0000  28.0000
969      eliwood  139.000000  139.000000  Default  27.0000  27.0000
1906      lyndis   97.000000   97.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
3456        want   12.424177   78.313590   Topic6  -5.1505   0.2462
1319       going   10.236496   72.990861   Topic6  -5.3442   0.1229
464       caelin    9.325140   69.043483   Topic6  -5.4374   0.0852
1147       fight    7.949788   51.065419   Topic6  -5.5970   0.2273
3343  understand    7.491136   45.578155   Topic6  -5.6564   0.2815

[398 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         4  0.914307        000
1         4  0.914307        100
22        1  0.152295  abilities
22        6  0.761475  abilities
65        6  0.915571     adrift
...     ...       ...        ...
3579      5  0.919080     wounds
3586      3  0.097682      wrong
3586      4  0.537252      wrong
3586      5  0.146523      wrong
3586      6  0.146523      wrong

[755 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 5, 4, 1, 3, 2])

1. Lord Eliwood trying to save Ninian
2. Assassination plot against Marquess Caelin
3. Lyn petitioning for help to reunite with her grandfather
4. Hector or Mark lending aid
5. Nils and Ninian trapped at Dragon's Gate
6. Miscellaneous

In [19]:

pyLDAvis.sklearn.prepare(lda_tfidf, X_tfidf, tfidf)

/Users/dayvdoberne/opt/anaconda3/envs/metis/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/dayvdoberne/opt/anaconda3/envs/metis/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.083870 -0.001734       1        1  18.441772
5     -0.073753 -0.081562       2        1  18.291506
3      0.006615  0.106963       3        1  16.978253
2      0.118217 -0.071861       4        1  16.382647
0     -0.009712  0.008074       5        1  16.311163
1      0.042503  0.040120       6        1  13.594660, topic_info=        Term       Freq      Total Category  logprob  loglift
3179   thank  29.000000  29.000000  Default  30.0000  30.0000
1869    lord  44.000000  44.000000  Default  29.0000  29.0000
2928   sorry  20.000000  20.000000  Default  28.0000  28.0000
1906  lyndis  30.000000  30.000000  Default  27.0000  27.0000
2103  ninian  27.000000  27.000000  Default  26.0000  26.0000
...      ...        ...        ...      ...      ...      ...
1925     man   2.729108  12.607711   Topic6  -5.8043   0.4652
464   caelin   2.671593  15.226706   Topic6  -5.8256   0.2551
1876    lost   2.326235   6.288987   Topic6  -5.9641   1.0009
2637   right   2.531563  38.349491   Topic6  -5.8795  -0.7224
1319   going   2.269914  20.572174   Topic6  -5.9886  -0.2087

[348 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         5  0.573748    000
1         5  0.573748    100
24        1  0.205364   able
24        3  0.616092   able
24        4  0.205364   able
...     ...       ...    ...
3572      4  0.947614  worst
3577      4  0.797560  wound
3586      4  0.131983  wrong
3586      5  0.659913  wrong
3586      6  0.131983  wrong

[618 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 4, 3, 1, 2])

In [ ]:
X